In [17]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
import pandas as pd
from src.gpt_prompt import get_sentiment_general, get_sentiment_general_parallel, get_risk_score
import datetime

In [18]:
df_news = pd.read_csv("../data/apple_2023/news_apple_2023.csv", index_col=0)
# df_news["date"] = pd.to_datetime(df_news['date'])

In [19]:
n = df_news.shape[0]
ind = 3

In [20]:
scores = []
batch = []
index = []
for i in range(n):
    if i % 100 == ind:
        index.append(i)
        batch.append(df_news["news"][i])
    # index.append(i)
    # batch.append(df_news["news"][i])
    if len(batch) >= 40:
        try:
            result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
            print(f"finish {i}")
            scores += result1
            batch = []  
        except:
            print(f"Fail {index[-1]}")
            index = index[:-100]
            batch = [] 
            
            
if batch:
    try:
        result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
        print(f"finish {i}")
        scores += result1
        batch = []  
    except:
        print(f"Fail {index[-1]}")

        
    
    
    

 average running time: 0.45 second
finish 903
 average running time: 1.89 second
finish 1903
 average running time: 0.52 second
finish 2903
 average running time: 0.40 second
finish 3903
 average running time: 0.39 second
finish 4903
 average running time: 0.35 second
finish 5903
 average running time: 0.46 second
finish 6903
 average running time: 0.32 second
finish 7903
 average running time: 0.42 second
finish 8903
 average running time: 0.39 second
finish 9903


In [ ]:
len(index)

332

In [ ]:
index = index[:len(scores)]
len(scores)

332

In [ ]:
def convert_response_to_score(response):
    dic = {"BULLISH": 1, "NEUTRAL": 0, "BEARISH":-1}
    scores = []
    for r in response:
        if r[0] not in dic:
            scores.append(5)
            continue
        scores.append(dic[r[0]])
        
    return scores

scores = convert_response_to_score(scores)
        

In [ ]:
new_df = df_news.iloc[index].copy()
new_df["score"] = scores
new_df = new_df.reset_index(drop=True)
new_df.to_csv(f"../data/apple_2023/news_score_apple_2023_{ind+1}.csv")

In [ ]:
new_df

,date,news,score
0,2023-01-01 01:32:30,Title: 2023 Dividend Growth Portfolio Review A...,1
1,2023-01-03 09:02:42,Title: Dividend Growth 50: Income In Year 8 Wa...,1
2,2023-01-03 19:02:25,Title: EXCLUSIVE: Top 10 Searched Tickers On B...,0
3,2023-01-04 08:58:25,Title: Wednesday’s Top Analyst Upgrades and Do...,0
4,2023-01-04 18:50:35,Title: Apple (AAPL) Outpaces Stock Market Gain...,1
...,...,...,...
327,2023-11-27 17:50:49,"Title: If this robot vacuum gets any cheaper, ...",1
328,2023-11-28 18:30:32,Title: Apple to end credit-card partnership wi...,-1
329,2023-11-29 14:45:33,Title: Arm gets minimal income from Apple desp...,-1
330,2023-11-30 10:50:15,"Title: Getting Closer: The Bloomberg Close, Eu...",0


In [ ]:
dictionary = new_df.set_index('date')['score'].to_dict()
dictionary

{'2023-01-01 01:32:30': 1,
 '2023-01-03 09:02:42': 1,
 '2023-01-03 19:02:25': 0,
 '2023-01-04 08:58:25': 0,
 '2023-01-04 18:50:35': 1,
 '2023-01-05 13:41:44': -1,
 '2023-01-06 05:53:29': -1,
 '2023-01-07 04:25:15': 1,
 '2023-01-09 06:49:53': 0,
 '2023-01-09 16:48:34': 1,
 '2023-01-10 09:44:16': -1,
 '2023-01-11 00:11:30': 0,
 '2023-01-11 12:47:29': 0,
 '2023-01-12 06:48:33': 0,
 '2023-01-12 17:12:13': -1,
 '2023-01-13 13:38:22': 1,
 '2023-01-15 09:12:46': 1,
 '2023-01-17 07:28:11': 0,
 '2023-01-17 14:21:42': 1,
 '2023-01-18 06:47:41': 0,
 '2023-01-18 14:13:30': 1,
 '2023-01-19 06:26:31': -1,
 '2023-01-19 17:06:57': 1,
 '2023-01-20 10:57:14': 1,
 '2023-01-22 08:03:32': 1,
 '2023-01-23 12:34:24': 1,
 '2023-01-23 19:36:02': -1,
 '2023-01-24 10:49:23': 0,
 '2023-01-25 04:07:35': 0,
 '2023-01-25 17:07:47': 1,
 '2023-01-26 13:00:37': 0,
 '2023-01-27 09:41:37': 1,
 '2023-01-28 10:28:59': -1,
 '2023-01-30 07:08:28': 0,
 '2023-01-31 06:58:05': 1,
 '2023-01-31 17:11:23': 1,
 '2023-02-01 09:33:45

In [ ]:
def find_whole_minute_time_after_5_minutes(time):
    new_time = given_time + timedelta(minutes=5)

    # Adjust to the nearest whole minute
    if new_time.second > 0:
        new_time += timedelta(minutes=1)
        new_time = new_time.replace(second=0, microsecond=0)

In [ ]:
# news = df["news"][0]
# print(get_sentiment_general(news,company="Apple",model="gpt-3.5-turbo-16k", output="polar", explanation=False)[0])
# print(get_sentiment_general(news,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False)[0])

In [ ]:
# news = df["news"][::400]
# result1 = get_sentiment_general_parallel(news,company="Apple",model="gpt-3.5-turbo-16k", output="polar", explanation=False)
# print(result1)
# result2 = get_sentiment_general_parallel(news,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False)
# print(result2)


In [ ]:
# n = len(result1)
# c = 0
# for i in range(n):
#     if result1[i] == result2[i]:
#         c += 1
# print(c / n)
        